## 개체명 인식
- **각 단어의 유형이 사람, 장소, 단체 등 어떤 유형인지를 알아내는 개체명 인식(Named Entity Recognition)**

<br>

- 개체명 인식(Named Entity Recognition)이란 말 그대로 이름을 가진 개체(named entity)를 인식

- 어떤 이름을 의미하는 단어를 보고는 그 단어가 어떤 유형인지를 인식하는 것

  유정 - 사람

  2018년 - 시간

  골드만삭스 - 조직

<br>

In [ ]:
pip install transformers

In [ ]:
pip install seqeval

In [ ]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from transformers import shape_list, BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from seqeval.metrics import f1_score, classification_report
import tensorflow as tf

<br>

### 데이터 로드

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1jatBP8yZkWn6Kg6mjN7nWLnYVwXE_sY_" -O ner_train_data.csv && rm -rf /tmp/cookies.txt

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1YVYShKCtWfigXBOb5ie7s6QmA-dHnjt3" -O ner_test_data.csv && rm -rf /tmp/cookies.txt

In [ ]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1_DPfdY1Q5Xt2md7QVbKcQLRDYHm3qgVQ" -O ner_label.txt && rm -rf /tmp/cookies.txt

In [ ]:
train_ner_df = pd.read_csv("ner_train_data.csv")

In [ ]:
train_ner_df.head()

,Sentence,Tag
0,"정은 씨를 힘들게 한 가스나그, 가만둘 수 없겠죠 .",PER-B O O O O O O O O
1,▶ 쿠마리 한동수가 말하는 '가넷 & 에르덴',O PER-B PER-I O PER-B O PER-B
2,슈나이더의 프레젠테이션은 말 청중을 위한 특별한 쇼다 .,PER-B O O CVL-B O O O O
3,지구 최대 연료탱크 수검 회사 구글이 연내 22명 안팎의 인력을 갖춘 연구개발(R&...,O O TRM-B O O ORG-B DAT-B NUM-B O O O ORG-B LO...
4,5. <10:00:TI_HOUR> 도이치증권대 <0:1:QT_SPORTS> 연예오락...,NUM-B O ORG-B O ORG-B


In [ ]:
test_ner_df = pd.read_csv("ner_test_data.csv")

In [ ]:
test_ner_df.head()

,Sentence,Tag
0,"라티은-원윤정, 휘닉스파크클래식 프로골퍼",PER-B EVT-B CVL-B
1,5원으로 맺어진 애인까지 돈이라는 민감한 원자재를 통해 현대인의 물질만능주의를 꼬집...,NUM-B O O O O O O O O O O O FLD-B O
2,-날로 삼키면 맛이 어떤지 일차 드셔보시겠어요 .,O O O O NUM-B O O
3,"-네, 지었습니다 .",O O O
4,◇신규 투자촉진에 방점=이번 접속료 조정결과에서 눈에 띄는 지점은 WCDMA/HSD...,O O O O O O O O TRM-B O TRM-B TRM-I ORG-B O TR...


- 훈련 데이터와 테스트 데이터의 샘플 개수를 확인

In [ ]:
print("학습 데이터 샘플 개수 :", len(train_ner_df))
print("테스트 데이터 샘플 개수 :", len(test_ner_df))

학습 데이터 샘플 개수 : 81000
테스트 데이터 샘플 개수 : 9000


In [ ]:
train_data_sentence = [sent.split() for sent in train_ner_df['Sentence'].values]
test_data_sentence = [sent.split() for sent in test_ner_df['Sentence'].values]
train_data_label = [tag.split() for tag in train_ner_df['Tag'].values]
test_data_label = [tag.split() for tag in test_ner_df['Tag'].values]

<br>

#### BIO 표현
- 개체명 인식에서 코퍼스로부터 개체명을 인식하기 위한 가장 보편적인 방법 중 하나인 BIO 태깅
  - B는 Begin의 약자로 개체명이 시작되는 부분
  - I는 Inside의 약자로 개체명의 내부 부분
  - O는 Outside의 약자로 개체명이 아닌 부분

In [ ]:
labels = [label.strip() for label in open('ner_label.txt', 'r', encoding='utf-8')]
print('개체명 태깅 정보 :', labels)

개체명 태깅 정보 : ['O', 'PER-B', 'PER-I', 'FLD-B', 'FLD-I', 'AFW-B', 'AFW-I', 'ORG-B', 'ORG-I', 'LOC-B', 'LOC-I', 'CVL-B', 'CVL-I', 'DAT-B', 'DAT-I', 'TIM-B', 'TIM-I', 'NUM-B', 'NUM-I', 'EVT-B', 'EVT-I', 'ANM-B', 'ANM-I', 'PLT-B', 'PLT-I', 'MAT-B', 'MAT-I', 'TRM-B', 'TRM-I']


In [ ]:
labels

['O',
 'PER-B',
 'PER-I',
 'FLD-B',
 'FLD-I',
 'AFW-B',
 'AFW-I',
 'ORG-B',
 'ORG-I',
 'LOC-B',
 'LOC-I',
 'CVL-B',
 'CVL-I',
 'DAT-B',
 'DAT-I',
 'TIM-B',
 'TIM-I',
 'NUM-B',
 'NUM-I',
 'EVT-B',
 'EVT-I',
 'ANM-B',
 'ANM-I',
 'PLT-B',
 'PLT-I',
 'MAT-B',
 'MAT-I',
 'TRM-B',
 'TRM-I']

In [ ]:
tag_to_index = {tag: index for index, tag in enumerate(labels)}
index_to_tag = {index: tag for index, tag in enumerate(labels)}

In [ ]:
tag_size = len(tag_to_index)
print('개체명 태깅 정보의 개수 :',tag_size)

개체명 태깅 정보의 개수 : 29


<br>

### 전처리
- BERT모델 로드

In [ ]:
tokenizer = BertTokenizer.from_pretrained("klue/bert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

<br>

- 2개의 입력 문장에 대한 어텐션 마스킹 & 세그먼트 임베딩

In [ ]:
def convert_examples_to_features(examples, labels, max_seq_len, tokenizer, pad_token_id_for_segment=0, pad_token_id_for_label=-100):
    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, data_labels = [], [], [], []

    for example, label in tqdm(zip(examples, labels), total=len(examples)):

        tokens = []
        labels_ids = []

        for one_word, label_token in zip(example, label):
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            labels_ids.extend([tag_to_index[label_token]]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            labels_ids = labels_ids[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        labels_ids += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        labels_ids = [pad_token_id_for_label] + labels_ids

        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)

        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label = labels_ids + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label) == max_seq_len, "Error with labels length {} vs {}".format(len(label), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        data_labels.append(label)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    data_labels = np.asarray(data_labels, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), data_labels

In [ ]:
X_train, y_train = convert_examples_to_features(train_data_sentence, train_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 81000/81000 [01:21<00:00, 993.77it/s] 


In [ ]:
X_test, y_test = convert_examples_to_features(test_data_sentence, test_data_label, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 9000/9000 [00:13<00:00, 684.40it/s]


<br>

#### TPU 설정 하에서의 모델 생성
- TPU 구동

In [ ]:
from transformers import TFBertForTokenClassification

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

strategy = tf.distribute.experimental.TPUStrategy(resolver)

- 모델 생성 및 컴파일


In [ ]:
with strategy.scope():
  model = TFBertForTokenClassification.from_pretrained("klue/bert-base", num_labels=tag_size, from_pt=True)
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  model.compile(optimizer=optimizer, loss=model.hf_compute_loss)

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForTokenClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForTokenClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForTokenClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForTokenClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


<br>

#### 모델 학습
- 각 에포크마다 F1-Score 평가

In [ ]:
class F1score(tf.keras.callbacks.Callback):
    def __init__(self, X_test, y_test):
        self.X_test = X_test
        self.y_test = y_test

    def sequences_to_tags(self, label_ids, pred_ids):
      label_list = []
      pred_list = []

      for i in range(0, len(label_ids)):
        label_tag = []
        pred_tag = []

        for label_index, pred_index in zip(label_ids[i], pred_ids[i]):
          if label_index != -100:
            label_tag.append(index_to_tag[label_index])
            pred_tag.append(index_to_tag[pred_index])

        label_list.append(label_tag)
        pred_list.append(pred_tag)

      return label_list, pred_list

    def on_epoch_end(self, epoch, logs={}):
      y_predicted = self.model.predict(self.X_test)
      y_predicted = np.argmax(y_predicted.logits, axis = 2)

      label_list, pred_list = self.sequences_to_tags(self.y_test, y_predicted)

      score = f1_score(label_list, pred_list, suffix=True)
      print(' - f1: {:04.2f}'.format(score * 100))
      print(classification_report(label_list, pred_list, suffix=True))

In [ ]:
f1_score_report = F1score(X_test, y_test)

In [ ]:
model.fit(
    X_train, y_train, epochs=3, batch_size=32,
    callbacks = [f1_score_report]
)

Epoch 1/3
   6/2532 [..............................] - ETA: 3:15 - loss: 2.2658

282/282 [==============================] - 20s 51ms/step
 - f1: 84.97
              precision    recall  f1-score   support

         AFW       0.61      0.63      0.62       394
         ANM       0.81      0.63      0.71       701
         CVL       0.82      0.83      0.83      5758
         DAT       0.92      0.92      0.92      2521
         EVT       0.74      0.77      0.75      1094
         FLD       0.52      0.71      0.60       228
         LOC       0.88      0.82      0.85      2126
         MAT       0.20      0.08      0.12        12
         NUM       0.92      0.92      0.92      5590
         ORG       0.86      0.86      0.86      4086
         PER       0.89      0.88      0.89      4426
         PLT       0.43      0.18      0.25        34
         TIM       0.78      0.90      0.84       314
         TRM       0.76      0.68      0.72      1964

   micro avg       0.85      0.84      0.85     29248
   macro avg       0.72      0.70      0.70     29248
weighted a

<br>

#### 예측
* 테스트 데이터에 대한 입력 전처리

In [ ]:
def convert_examples_to_features_for_prediction(examples, max_seq_len, tokenizer, pad_token_id_for_segment=0, pad_token_id_for_label=-100):

    cls_token = tokenizer.cls_token
    sep_token = tokenizer.sep_token
    pad_token_id = tokenizer.pad_token_id

    input_ids, attention_masks, token_type_ids, label_masks = [], [], [], []

    for example in tqdm(examples):
        tokens = []
        label_mask = []
        for one_word in example:
            subword_tokens = tokenizer.tokenize(one_word)
            tokens.extend(subword_tokens)
            label_mask.extend([0]+ [pad_token_id_for_label] * (len(subword_tokens) - 1))

        special_tokens_count = 2
        if len(tokens) > max_seq_len - special_tokens_count:
            tokens = tokens[:(max_seq_len - special_tokens_count)]
            label_mask = label_mask[:(max_seq_len - special_tokens_count)]

        tokens += [sep_token]
        label_mask += [pad_token_id_for_label]
        tokens = [cls_token] + tokens
        label_mask = [pad_token_id_for_label] + label_mask
        input_id = tokenizer.convert_tokens_to_ids(tokens)
        attention_mask = [1] * len(input_id)
        padding_count = max_seq_len - len(input_id)
        input_id = input_id + ([pad_token_id] * padding_count)
        attention_mask = attention_mask + ([0] * padding_count)
        token_type_id = [pad_token_id_for_segment] * max_seq_len
        label_mask = label_mask + ([pad_token_id_for_label] * padding_count)

        assert len(input_id) == max_seq_len, "Error with input length {} vs {}".format(len(input_id), max_seq_len)
        assert len(attention_mask) == max_seq_len, "Error with attention mask length {} vs {}".format(len(attention_mask), max_seq_len)
        assert len(token_type_id) == max_seq_len, "Error with token type length {} vs {}".format(len(token_type_id), max_seq_len)
        assert len(label_mask) == max_seq_len, "Error with labels length {} vs {}".format(len(label_mask), max_seq_len)

        input_ids.append(input_id)
        attention_masks.append(attention_mask)
        token_type_ids.append(token_type_id)
        label_masks.append(label_mask)

    input_ids = np.array(input_ids, dtype=int)
    attention_masks = np.array(attention_masks, dtype=int)
    token_type_ids = np.array(token_type_ids, dtype=int)
    label_masks = np.asarray(label_masks, dtype=np.int32)

    return (input_ids, attention_masks, token_type_ids), label_masks

In [ ]:
X_pred, label_masks = convert_examples_to_features_for_prediction(test_data_sentence[:5], max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 5/5 [00:00<00:00, 800.01it/s]


<br>

- 개체명 인식 확인

In [ ]:
def ner_prediction(examples, max_seq_len, tokenizer):

  examples = [sent.split() for sent in examples]
  X_pred, label_masks = convert_examples_to_features_for_prediction(examples, max_seq_len=128, tokenizer=tokenizer)
  y_predicted = model.predict(X_pred)
  y_predicted = np.argmax(y_predicted.logits, axis = 2)

  pred_list = []
  result_list = []

  for i in range(0, len(label_masks)):
    pred_tag = []
    for label_index, pred_index in zip(label_masks[i], y_predicted[i]):
      if label_index != -100:
        pred_tag.append(index_to_tag[pred_index])

    pred_list.append(pred_tag)

  for example, pred in zip(examples, pred_list):
    one_sample_result = []
    for one_word, label_token in zip(example, pred):
      one_sample_result.append((one_word, label_token))
    result_list.append(one_sample_result)

  return result_list

In [ ]:
sent1 = '오리온스는 리그 최정상급 포인트가드 김동훈을 앞세우는 빠른 공수전환이 돋보이는 팀이다'
sent2 = '하이신사에 속한 섬들도 위로 솟아 있는데 타인은 살고 있어요'

In [ ]:
test_samples = [sent1, sent2]

In [ ]:
result_list = ner_prediction(test_samples, max_seq_len=128, tokenizer=tokenizer)

100%|██████████| 2/2 [00:00<00:00, 809.55it/s]


1/1 [==============================] - 1s 638ms/step


In [ ]:
result_list

[[('오리온스는', 'ORG-B'),
  ('리그', 'O'),
  ('최정상급', 'O'),
  ('포인트가드', 'CVL-B'),
  ('김동훈을', 'PER-B'),
  ('앞세우는', 'O'),
  ('빠른', 'O'),
  ('공수전환이', 'O'),
  ('돋보이는', 'O'),
  ('팀이다', 'O')],
 [('하이신사에', 'LOC-B'),
  ('속한', 'O'),
  ('섬들도', 'O'),
  ('위로', 'O'),
  ('솟아', 'O'),
  ('있는데', 'O'),
  ('타인은', 'O'),
  ('살고', 'O'),
  ('있어요', 'O')]]

<br>

## KoBERT를 이용한 기계 독해(Machine Reading Comprehension)
- **KorQuad 1.0 데이터를 학습하여 사용자의 질문으로부터 본문으로부터 정답을 찾아 대답하는 모델을 구현**

<br>

#### 데이터 로드

In [ ]:
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_train.json -O KorQuAD_v1.0_train.json
!wget https://korquad.github.io/dataset/KorQuAD_v1.0_dev.json -O KorQuAD_v1.0_dev.json

In [2]:
import os
import json
import numpy as np
from tqdm import tqdm
from pathlib import Path
from transformers import BertTokenizerFast
import tensorflow as tf

In [16]:
path = 'KorQuAD_v1.0_train.json'
with open(path, 'rb') as f:
  squad_dict = json.load(f)

<br>

- 각 본문(context)와 본문에 대한 질문(Question)과 답변(Answer)이 포함된 json
- 각 답변에는 해당 답변이 본문에서 가지는 위치 인덱스 (answer_start)가 포함

In [20]:
squad_dict['data'][0]

{'paragraphs': [{'qas': [{'answers': [{'text': '교향곡', 'answer_start': 54}],
     'id': '6566495-0-0',
     'question': '바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?'},
    {'answers': [{'text': '1악장', 'answer_start': 421}],
     'id': '6566495-0-1',
     'question': '바그너는 교향곡 작곡을 어디까지 쓴 뒤에 중단했는가?'},
    {'answers': [{'text': '베토벤의 교향곡 9번', 'answer_start': 194}],
     'id': '6566495-0-2',
     'question': '바그너가 파우스트 서곡을 쓸 때 어떤 곡의 영향을 받았는가?'},
    {'answers': [{'text': '파우스트', 'answer_start': 15}],
     'id': '6566518-0-0',
     'question': '1839년 바그너가 교향곡의 소재로 쓰려고 했던 책은?'},
    {'answers': [{'text': '합창교향곡', 'answer_start': 354}],
     'id': '6566518-0-1',
     'question': '파우스트 서곡의 라단조 조성이 영향을 받은 베토벤의 곡은?'},
    {'answers': [{'text': '1839', 'answer_start': 0}],
     'id': '5917067-0-0',
     'question': '바그너가 파우스트를 처음으로 읽은 년도는?'},
    {'answers': [{'text': '파리', 'answer_start': 410}],
     'id': '5917067-0-1',
     'question': '바그너가 처음 교향곡 작곡을 한 장소는?'},
    {'answers': [{'text': '드레스덴', 'answer_sta

In [3]:
def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

<br>

- **본문(context), 질문(question), 답변(answer) 데이터 확인**

In [4]:
train_contexts, train_questions, train_answers = read_squad('KorQuAD_v1.0_train.json')
val_contexts, val_questions, val_answers = read_squad('KorQuAD_v1.0_dev.json')

In [5]:
print('훈련 데이터의 본문 개수 :', len(train_contexts))
print('훈련 데이터의 질문 개수 :', len(train_questions))
print('훈련 데이터의 답변 개수 :', len(train_answers))
print('테스트 데이터의 본문 개수 :', len(val_contexts))
print('테스트 데이터의 질문 개수 :', len(val_questions))
print('테스트 데이터의 답변 개수 :', len(val_answers))

훈련 데이터의 본문 개수 : 60407
훈련 데이터의 질문 개수 : 60407
훈련 데이터의 답변 개수 : 60407
테스트 데이터의 본문 개수 : 5774
테스트 데이터의 질문 개수 : 5774
테스트 데이터의 답변 개수 : 5774


In [6]:
print('첫번째 샘플의 본문')
print('-----------------')
print(train_contexts[0])

첫번째 샘플의 본문
-----------------
1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡(1악장)을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다.


In [ ]:
train_contexts[0]

In [7]:
print('첫번째 샘플의 질문')
print('-----------------')
print(train_questions[0])

첫번째 샘플의 질문
-----------------
바그너는 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가?


In [8]:
print('첫번째 샘플의 답변')
print('-----------------')
print(train_answers[0])

첫번째 샘플의 답변
-----------------
{'text': '교향곡', 'answer_start': 54}


<br>

- 답변 데이터에 답변이 본문에서 가지는 위치 시작 인덱스와 종료 인덱스를 추가

In [9]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        answer['text'] = answer['text'].rstrip()
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        assert context[start_idx:end_idx] == gold_text, "end_index 계산에 에러가 있습니다."
        answer['answer_end'] = end_idx

In [10]:
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [12]:
train_answers[:5]

[{'text': '교향곡', 'answer_start': 54, 'answer_end': 57},
 {'text': '1악장', 'answer_start': 421, 'answer_end': 424},
 {'text': '베토벤의 교향곡 9번', 'answer_start': 194, 'answer_end': 205},
 {'text': '파우스트', 'answer_start': 15, 'answer_end': 19},
 {'text': '합창교향곡', 'answer_start': 354, 'answer_end': 359}]

<br>

#### BERT 모델 로드 및 토크나이징

In [21]:
tokenizer = BertTokenizerFast.from_pretrained('klue/bert-base')

train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

- 각 인코딩에 질문의 시작/종료 인덱스값 추가

In [22]:
def add_token_positions(encodings, answers):

    start_positions = []
    end_positions = []
    deleting_list = []

    for i in tqdm(range(len(answers))):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end'] - 1))

        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length
            deleting_list.append(i)

        if end_positions[-1] is None:
            end_positions[-1] = tokenizer.model_max_length
            if i not in deleting_list:
              deleting_list.append(i)

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})
    return deleting_list

In [23]:
deleting_list_for_train = add_token_positions(train_encodings, train_answers)
deleting_list_for_test = add_token_positions(val_encodings, val_answers)

100%|██████████| 5774/5774 [00:00<00:00, 237467.75it/s]


In [24]:
def delete_samples(encodings, deleting_list):
  input_ids = np.delete(np.array(encodings['input_ids']), deleting_list, axis=0)
  attention_masks = np.delete(np.array(encodings['attention_mask']), deleting_list, axis=0)
  start_positions = np.delete(np.array(encodings['start_positions']), deleting_list, axis=0)
  end_positions = np.delete(np.array(encodings['end_positions']), deleting_list, axis=0)

  X_data = [input_ids, attention_masks]
  y_data = [start_positions, end_positions]

  return X_data, y_data

In [28]:
X_train, y_train = delete_samples(train_encodings, deleting_list_for_train)
X_test, y_test = delete_samples(val_encodings, deleting_list_for_test)

<br>

#### TPU 구동

In [29]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.config.experimental_connect_to_cluster(resolver)
tf.tpu.experimental.initialize_tpu_system(resolver)

In [30]:
strategy = tf.distribute.TPUStrategy(resolver)

In [31]:
from transformers import TFBertModel

<br>

#### 모델에 출력층 추가

In [32]:
class TFBertForQuestionAnswering(tf.keras.Model):
    def __init__(self, model_name):
        super(TFBertForQuestionAnswering, self).__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.qa_outputs = tf.keras.layers.Dense(2,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                name='qa_outputs')
        self.softmax = tf.keras.layers.Activation(tf.keras.activations.softmax)

    def call(self, inputs):
        input_ids, attention_mask = inputs
        outputs = self.bert(input_ids, attention_mask=attention_mask)

        sequence_output = outputs[0]

        logits = self.qa_outputs(sequence_output)
        start_logits, end_logits = tf.split(logits, 2, axis=-1)

        # start_logits = (batch_size, sequence_length,)
        # end_logits = (batch_size, sequence_length,)
        start_logits = tf.squeeze(start_logits, axis=-1)
        end_logits = tf.squeeze(end_logits, axis=-1)

        start_probs = self.softmax(start_logits)
        end_probs = self.softmax(end_logits)

        return start_probs, end_probs

<br>

#### 모델 생성 및 컴파일

In [ ]:
with strategy.scope():
  model = TFBertForQuestionAnswering("klue/bert-base")
  optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
  model.compile(optimizer=optimizer, loss=loss)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

<br>

#### 모델 적합

In [ ]:
history = model.fit(
    X_train,
    y_train,
    epochs=3,
    verbose=1,
    batch_size=16,
)

Epoch 1/3


3759/3759 [==============================] - 537s 117ms/step - loss: 1.1286 - output_1_loss: 0.5070 - output_2_loss: 0.6217
Epoch 2/3
3759/3759 [==============================] - 442s 118ms/step - loss: 0.6167 - output_1_loss: 0.2628 - output_2_loss: 0.3539
Epoch 3/3
3759/3759 [==============================] - 441s 117ms/step - loss: 0.4489 - output_1_loss: 0.1848 - output_2_loss: 0.2641


<br>

#### 모델 평가

In [ ]:
def predict_test_data_by_idx(idx):
  context = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[0]
  question = tokenizer.decode(X_test[0][idx]).split('[SEP] ')[1]
  print('본문 :', context)
  print('질문 :', question)
  answer_encoded = X_test[0][idx][y_test[0][idx]:y_test[1][idx]+1]
  print('정답 :',tokenizer.decode(answer_encoded))
  output = model([tf.constant(X_test[0][idx])[None, :], tf.constant(X_test[1][idx])[None, :]])
  start = tf.math.argmax(tf.squeeze(output[0]))
  end = tf.math.argmax(tf.squeeze(output[1]))+1
  answer_encoded = X_test[0][idx][start:end]
  print('예측 :',tokenizer.decode(answer_encoded))
  print('----------------------------------------')

In [ ]:
for i in range(0, 100):
  predict_test_data_by_idx(i)

본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학생회관 건물 계단을 내려오는 임종석을 발견, 검거해 구속을 집행했다. 임종석은 청량리경찰서에서 약 1시간 동안 조사를 받은 뒤 오전 9시 50분 경 서울 장안동의 서울지방경찰청 공안분실로 인계되었다. 
질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은? 
정답 : 1989년 2월 15일
예측 : 1989년 2월 15일
----------------------------------------
본문 : [CLS] 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ( 폭력행위등처벌에관한법률위반 ) 으로 지명수배되었다. 1989년 3월 12일 서울지방검찰청 공안부는 임종석의 사전구속영장을 발부받았다. 같은 해 6월 30일 평양축전에 임수경을 대표로 파견하여 국가보안법위반 혐의가 추가되었다. 경찰은 12월 18일 ~ 20일 사이 서울 경희대학교에서 임종석이 성명 발표를 추진하고 있다는 첩보를 입수했고, 12월 18일 오전 7시 40분 경 가스총과 전자봉으로 무장한 특공조 및 대공과 직원 12명 등 22명의 사복 경찰을 승용차 8대에 나누어 경희대학교에 투입했다. 1989년 12월 18일 오전 8시 15분 경 서울청량리경찰서는 호위 학생 5명과 함께 경희대학교 학

<br>

## BERT의 문장 임베딩(SBERT)을 이용한 한국어 챗봇
- SBERT를 이용하여 문장 임베딩을 얻을 수 있는 패키지인 `sentence_transformers`를 사용하여 쉽고 간단하게 한국어 챗봇을 구현

In [ ]:
pip install sentence_transformers

In [2]:
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
from sentence_transformers import SentenceTransformer

<br>

#### 데이터 로드

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


<br>

#### BERT 모델 로드 및 인코딩
- 한국어도 포함되어 학습된 다국어 모델을 로드
- 모델의 이름은 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens':
  - 이름이 의미하는 바는 100가지 언어를 지원(한국어 포함)하는 다국어 BERT BASE 모델
  - SNLI 데이터를 학습 후 STS-B 데이터로 학습되었으며, 문장 표현을 얻기 위해서는 평균 풀링(mean-tokens)을 사용했다는 의미
  
    $→$ NLI 데이터를 학습 후에 STS 데이터로 추가 파인 튜닝한 모델이라는 의미입니다.

- `SentenceTransformer`로 로드할 수 있는 다양한 모델에 대한 리스트는 아래의 링크에서 확인 가능

  https://huggingface.co/models?library=sentence-transformers



In [4]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


<br>

- **데이터에서 모든 질문열. 즉, `train_data['Q']`에 대해서 문장 임베딩 값을 구한 후 embedding이라는 새로운 열에 저장**

In [5]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [6]:
train_data.head()

,Q,A,label,embedding
0,12시 땡!,하루가 또 가네요.,0,"[0.20179585, -0.03443809, 1.5395724, 0.0106974..."
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.077166, -0.03427817, 0.86244243, 0.02636061..."
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.10445247, -0.012432256, 1.0132879, 0.022501..."
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760747, -0.046716884, 0.8936941, 0.021047..."
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002865, 0.031961102, 1.4915428, 4.338047..."


<br>

- 두 개의 벡터로부터 코사인 유사도를 구하는 함수를 정의



In [7]:
import numpy as np
from numpy import dot
from numpy.linalg import norm

<br>

- `return_similar_answer` 함수는 임의의 질문이 들어오면 해당 질문의 문장 임베딩 값과 챗봇 데이터의 임베딩 열.
  
  즉, `train_data['embedding']`에 저장해둔 모든 질문 샘플들의 문장 임베딩 값들을 전부 비교하여 코사인 유사도 값이 가장 높은 질문 샘플을 찾아냄
  
  $\rightarrow$ 해당 질문 샘플과 짝이 되는 답변 샘플을 리턴


In [8]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [9]:
def return_similar_answer(input):
    embedding = model.encode(input)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    return train_data.loc[train_data['score'].idxmax()]['A']

- 테스트

In [10]:
return_similar_answer('결혼하고싶어')

'좋은 사람이랑 결혼할 수 있을 거예요.'

In [11]:
return_similar_answer('나랑 커피먹을래?')

'카페인이 필요한 시간인가 봐요.'

In [12]:
return_similar_answer('반가워')

'저도 반가워요.'

In [13]:
return_similar_answer('너는 누구니?')

'저는 위로봇입니다.'

In [14]:
return_similar_answer('영화')

'저도 영화 보여주세요.'

In [15]:
return_similar_answer('너무 짜증나')

'짜증날 땐 짜장면'

In [16]:
return_similar_answer('화가납니다')

'화를 참는 연습을 해보세요.'

In [17]:
return_similar_answer('나랑 놀자')

'지금 그러고 있어요.'

In [18]:
return_similar_answer('나랑 게임하자')

'같이 놀아요.'

In [19]:
return_similar_answer('출근하기싫어')

'씻고 푹 쉬세요.'

In [20]:
return_similar_answer('여행가고싶다')

'이김에 떠나보세요.'

In [21]:
return_similar_answer('너 말 잘한다')

'그런 사람이 있으면 저 좀 소개시켜주세요.'

<br>

## Faiss와 SBERT를 이용한 시맨틱 검색기(Semantic Search)
- **시맨틱 검색(Semantic search)은 기존의 키워드 매칭이 아닌 문장의 의미에 초점을 맞춘 정보 검색 시스템**
- SBERT와 FAISS를 사용하여 간단한 검색 엔진을 구현

<br>

#### Faiss
- Faiss는 벡터화 된 데이터를 인덱싱하고 데이터에 대한 효율적인 검색을 수행하기 위해 Facebook AI에서 구축한 C ++ 기반 라이브러리
  
  (CPU 환경이라면 faiss-gpu가 아니라 faiss-cpu를 설치)


In [ ]:
!pip install faiss-gpu
!pip install -U sentence-transformers

In [23]:
import numpy as np
import os
import pandas as pd
import urllib.request
import faiss
import time
from sentence_transformers import SentenceTransformer

<br>

#### 데이터 로드
- 여기서는 약 100만개의 뉴스 기사 제목 데이터를 사용
* 데이터를 로드하여 리스트 형태로 변환


In [24]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/ukairia777/tensorflow-nlp-tutorial/main/19.%20Topic%20Modeling%20(LDA%2C%20BERT-Based)/dataset/abcnews-date-text.csv", filename="abcnews-date-text.csv")

df = pd.read_csv("abcnews-date-text.csv")
data = df.headline_text.to_list()


In [29]:
data[:5]

['aba decides against community broadcasting licence',
 'act fire witnesses must be aware of defamation',
 'a g calls for infrastructure protection summit',
 'air nz staff in aust strike for pay rise',
 'air nz strike to affect australian travellers']

In [26]:
print('총 샘플의 개수 :', len(data))

총 샘플의 개수 : 1082168


<br>

#### SBERT 임베딩

In [30]:
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
encoded_data = model.encode(data[:100])
print('임베딩 된 벡터 수 :', len(encoded_data))

임베딩 된 벡터 수 : 100


<br>

#### 인덱스 정의 및 데이터 추가
- 인덱스를 정의하고 여기에 데이터를 추가

In [32]:
index = faiss.IndexIDMap(faiss.IndexFlatIP(768))
index.add_with_ids(encoded_data, np.array(range(0, 100)))

In [33]:
faiss.write_index(index, 'abc_news')

<br>

#### 검색 및 시간 측정
- 주어진 쿼리에 대해서 유사도가 높은 상위 5개의 샘플을 추출
* 약 108만개의 문서에 대해서 시맨틱 검색을 수행하였음에도 약 1초 내외의 시간

In [34]:
def search(query):
   t = time.time()
   query_vector = model.encode([query])
   k = 5
   top_k = index.search(query_vector, k)
   print('total time: {}'.format(time.time() - t))
   return [data[_id] for _id in top_k[1].tolist()[0]]

In [35]:
query = str(input())
results = search(query)

print('results :')
for result in results:
   print('\t', result)

Underwater Forest Discovered
total time: 0.05321311950683594
results :
	 investigation underway into elster creek spill
	 griffiths under fire over project knock back
	 antic delighted with record breaking barca
	 meeting to focus on broken hill water woes
	 massive drug crop discovered in western nsw
